<a href="https://colab.research.google.com/github/rassamyjs/DS-Unit-1-Build/blob/master/Copy_of_LS_DS_214_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [25]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [26]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [27]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [28]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [29]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [30]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421 entries, 0 to 422
Data columns (total 59 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Burrito         421 non-null    object        
 1   Date            421 non-null    datetime64[ns]
 2   Yelp            87 non-null     float64       
 3   Google          87 non-null     float64       
 4   Chips           26 non-null     object        
 5   Cost            414 non-null    float64       
 6   Hunger          418 non-null    float64       
 7   Mass (g)        22 non-null     float64       
 8   Density (g/mL)  22 non-null     float64       
 9   Length          283 non-null    float64       
 10  Circum          281 non-null    float64       
 11  Volume          281 non-null    float64       
 12  Tortilla        421 non-null    float64       
 13  Temp            401 non-null    float64       
 14  Meat            407 non-null    float64       
 15  Fillin

In [ ]:
df.head()

In [93]:
# Date to year/month column to datetime & drop Date
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
type(df['Year'][0])
df = df.drop(columns='Date')

In [ ]:
df.head()

*  Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.

In [95]:
# training data
train = df[df['Year'] <= 2016]
validate = df[df['Year'] == 2017]
test = df[df['Year'] >= 2018]


In [116]:
train.shape, validate.shape, test.shape, df.shape

((298, 60), (85, 60), (38, 60), (421, 60))

In [100]:
train['Great'].value_counts(normalize=True)

False    0.590604
True     0.409396
Name: Great, dtype: float64

In [97]:
#Split target vector from feature matrix
target = 'Great'
y = train[target]
X = train.drop(columns=target)
y1 = validate[target]
X1 = validate.drop(columns=target)


In [104]:
# training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_validate, X_val1, y_validate, y_val1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

- Is this a regression or classification
- This is going off of accuaracy = classification

In [106]:
# Establish Baseline
print('Baseline accuracy:', y_train.value_counts(normalize=True).max())


Baseline accuracy: 0.5840336134453782


# Build Our `LogisticRegression` Model

In [107]:
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [111]:
model_log = make_pipeline(
       OneHotEncoder(use_cat_names=True),
       SimpleImputer(),
       StandardScaler(),
       LogisticRegression()
)

model_log.fit(X_train, y_train);


/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [112]:
# logistic metric
print('Training Accuracy Score:', model_log.score(X_train, y_train))
print('Validation Accuracy Score:', model_log.score(X_val, y_val))

Training Accuracy Score: 0.9369747899159664
Validation Accuracy Score: 0.9333333333333333


In [113]:
# model for validate 
model_log1 = make_pipeline(
       OneHotEncoder(use_cat_names=True),
       SimpleImputer(),
       StandardScaler(),
       LogisticRegression()
)

model_log1.fit(X_validate, y_validate);


/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [114]:
# validate metric
print('Training Accuracy Score:', model_log1.score(X_validate, y_validate))
print('Validation Accuracy Score:', model_log1.score(X_val1, y_val1))

Training Accuracy Score: 0.9852941176470589
Validation Accuracy Score: 0.7058823529411765


# Get your model's test accuracy. (One time, at the end.)

In [122]:
test1 = test 
test1 = test1.drop(columns='Great')
print(test1.shape)

(38, 59)


In [125]:
# test model
y_pred = model_log.predict(test1)
# convert to dataframe
convert = pd.DataFrame(y_pred, columns=['Great'])
convert

,Great
0,True
1,True
2,False
3,True
4,False
5,False
6,True
7,True
8,True
9,True
